In [30]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import jupyter_util as util
from importlib import reload

In [73]:
reload(util)

<module 'jupyter_util' from '/home/sevni/Documents/thclab/aries/aries-acapy-plugin/utilities/jupyter_util.py'>

## Enter Invite Url:

In [32]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiODg5NjJjMjItNjEyMC00ODhiLWI5MzYtN2E0YjFjMTU1ODUyIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQxLmxvY2FsaG9zdCIsICJsYWJlbCI6ICJNYWluIChhZG1pbikiLCAicmVjaXBpZW50S2V5cyI6IFsiRm1LUnpFUWpIZnBaTlNqYWRiOU1IdWZZWnNTTXh6c2dCOW42VXJ0WmlXQiJdfQ==


In [33]:
agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiZjllZjgzM2YtZDJjZC00ODkwLTg2NDItMDI2NGRkOWUwZTZkIiwgInJlY2lwaWVudEtleXMiOiBbIkZpWkxkUm5GN0NVSFE4SmpKYjNITWZwUms5cTlYTXhyUVU4RkdZNzJnNlNrIl0sICJsYWJlbCI6ICJDbGllbnQgKGFkbWluKSIsICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovL2FnZW50Mi5sb2NhbGhvc3QifQ==


In [34]:
agent1 = util.processInviteUrl(agent1['url'])
agent2 = util.processInviteUrl(agent2['url'])
print("Decoded invitation:\n")
pp.pprint(agent2['invite'])

Decoded invitation:

{'@id': 'f9ef833f-d2cd-4890-8642-0264dd9e0e6d',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['FiZLdRnF7CUHQ8JjJb3HMfpRk9q9XMxrQU8FGY72g6Sk'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [35]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 4fb6eosjBij42tZrs33BX7L1ZLNiu3mi99yQjaWh8p5kGpdxiKEbE1zsBfS7Gi5UXf7HwBQ53CWUJAZ2VL6DysXL

Public key: 
 9t81otewmZj9n99BsYx32ZYPyMyi7rEEBufKneUQDe62


**Sending a connection request to acapy**

In [36]:
   
agent1['connection'] = util.connectWithAcapy(agent1, plugin)
agent2['connection'] = util.connectWithAcapy(agent2, plugin)
# print("Decoded acapy response: \n")
# pp.pprint(agent1['conn'])

**Unpacking connection data embedded in the response**

In [37]:
agent1['connection_details'] = util.decodeConnectionDetails(agent1['connection'])
agent2['connection_details'] = util.decodeConnectionDetails(agent2['connection'])

**Adding the connection to connection list**

In [38]:
connections = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": agent1['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
    {
        "label": agent2['invite']['label'],
        "DIDDoc": agent2['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
]

# pp.pprint(connections)

### FEATURE DISCOVERY

In [44]:
# pp.pprint(util.getFeatureDiscovery(connections[0]))

### Notes

* To receive invitation use "new connections" protocol "receive invitation"
* To create invitation use create invitation from "invitations" protocol

### Done
* create invitation
* receive invitation
* basic message
    * get
    * send
    * basic message
    * deletes
* dids
    * list dids
    

### Todo
* basic message sorting
* basic message delete

In [45]:
pp.pprint(util.connectAgents(connections[0], connections[1]))

{'@id': '8f376685-a89e-4e64-bb38-f4fc44d288a3',
 '@type': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-connections/0.1/connection',
 'connection_id': '55d7fc39-c40f-422b-b75d-b1f9c98bc114',
 'label': 'ConnectionBetweenAgents',
 'my_did': 'Njs1G7cpen9MKMrx1VTWT5',
 'raw_repr': {'accept': 'auto',
              'connection_id': '55d7fc39-c40f-422b-b75d-b1f9c98bc114',
              'created_at': '2020-06-03 09:52:25.375806Z',
              'initiator': 'external',
              'invitation_key': 'tcH5kgbkXTS4Rxm4zgcT2avz8SbjmRrpi6coXpvJLuj',
              'invitation_mode': 'once',
              'my_did': 'Njs1G7cpen9MKMrx1VTWT5',
              'request_id': 'dc32ff07-93bb-4181-bed4-b6181c5cf4ce',
              'routing_state': 'none',
              'state': 'request',
              'their_label': 'ConnectionBetweenAgents',
              'updated_at': '2020-06-03 09:52:25.387945Z'},
 'state': 'pending'}


In [47]:
pp.pprint(util.getConnectionList(connections[1]))

{'@id': '88ba6222-b1b0-4309-8f70-745684bd4e23',
 '@type': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-connections/0.1/list',
 'connections': [{'connection_id': '31e42ba3-99e8-4404-9003-9a115a4f7153',
                  'label': None,
                  'my_did': None,
                  'raw_repr': {'accept': 'auto',
                               'connection_id': '31e42ba3-99e8-4404-9003-9a115a4f7153',
                               'created_at': '2020-06-03 09:41:11.837134Z',
                               'initiator': 'self',
                               'invitation_key': '4gi4VNnXAg7zgcPiFi1SjcgENsEMSnVnDs9ii2BBSXND',
                               'invitation_mode': 'once',
                               'routing_state': 'none',
                               'state': 'invitation',
                               'their_role': 'admin',
                               'updated_at': '2020-06-03 09:41:11.837134Z'},
                  'role': 'admin',
            

In [49]:
pp.pprint(util.sendBasicMessage(connections[1], "55d7fc39-c40f-422b-b75d-b1f9c98bc114", "test delete"))

{'@id': '09f8a881-f817-40bc-8701-29c772db4b86',
 '@type': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-basicmessage/0.1/sent',
 'connection_id': '55d7fc39-c40f-422b-b75d-b1f9c98bc114',
 'message': {'connection_id': '55d7fc39-c40f-422b-b75d-b1f9c98bc114',
             'content': 'test delete',
             'message_id': 'f58c2401-2c67-42ae-a49a-71545273e389',
             'sent_time': '2020-06-03T10:04:05Z',
             'state': 'sent'},
 '~thread': {'thid': '3afddc9b-2920-433a-9ffe-b0b7c75c2f3b'}}


In [60]:
pp.pprint(util.getBasicMessages(connections[0]))

{'@id': '19646718-5848-431d-8a67-64866edce7b4',
 '@type': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-basicmessage/0.1/messages',
 'count': 0,
 'messages': [],
 'offset': 0,
 'remaining': 0,
 '~thread': {'thid': '34c4e176-154f-43b8-9072-e4f650cea317'}}


In [59]:
pp.pprint(deleteBasicMessage(connections[0], "5c637f43-b573-438d-a076-ff098e708893", 'f58c2401-2c67-42ae-a49a-71545273e389'))

{'@id': '3c649c8a-c567-4bb2-9ca8-aa8c81440d52',
 '@type': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-basicmessage/0.1/deleted',
 'connection_id': '5c637f43-b573-438d-a076-ff098e708893',
 'deleted': [{'connection_id': '5c637f43-b573-438d-a076-ff098e708893',
              'content': 'test delete',
              'message_id': 'f58c2401-2c67-42ae-a49a-71545273e389',
              'sent_time': '2020-06-03T10:04:05Z',
              'state': 'recv'}],
 '~thread': {'thid': '31b101d7-8e5e-4ebe-8849-63af5c0851ed'}}


In [62]:
def createDid(agent):
    message = util.buildMessage(
        "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/create-did"
    )
    return util.sendMessage(message, agent)

pp.pprint(createDid(connections[0]))

{'@id': '20e70536-5cf2-4203-b7f7-874144802887',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 'result': {'did': '7JQ2X2Gwh2p5ZZivz4kiev',
            'verkey': '4S9vLXq5FRw56brvM4dqH2UENNrmWbYNbucfhJFaEcWz'},
 '~thread': {'thid': 'b0af7280-950e-409f-8157-e27be829904a'}}


In [66]:
def setPublicDid(agent, did):
    message = util.buildMessage(
        "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/set-public-did",
        did=did
    )
    return util.sendMessage(message, agent)

pp.pprint(setPublicDid(connections[0], "7JQ2X2Gwh2p5ZZivz4kiev"))

{'@id': '2cae5cb4-d619-4f21-9186-1ff5bbd8eb88',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 'result': {'did': '7JQ2X2Gwh2p5ZZivz4kiev',
            'metadata': {'public': True},
            'verkey': '4S9vLXq5FRw56brvM4dqH2UENNrmWbYNbucfhJFaEcWz'},
 '~thread': {'thid': 'fd23b2b1-963c-4ee3-847e-8e9b3ba21d5a'}}


In [67]:
def getPublicDid(agent):
    message = util.buildMessage(
        "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-public-did"
    )
    return util.sendMessage(message, agent)

pp.pprint(getPublicDid(connections[0]))

{'@id': '3a171dc5-1b32-409f-a111-a69533e8b84c',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 'result': {'did': '7JQ2X2Gwh2p5ZZivz4kiev',
            'metadata': {'public': True},
            'verkey': '4S9vLXq5FRw56brvM4dqH2UENNrmWbYNbucfhJFaEcWz'},
 '~thread': {'thid': 'a7613921-8ad2-49e2-a90c-c992119c6e4d'}}


In [69]:
# this works but is not handled well on toolbox side
def registerDidOnLedger(agent,verkey,alias=None,role=None):
    message = util.buildMessage(
        "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/register-did",
        did=did,
        verkey=verkey
    )
    return util.sendMessage(message, agent

pp.pprint(registerDidOnLedger(connections[0], "7JQ2X2Gwh2p5ZZivz4kiev", "4S9vLXq5FRw56brvM4dqH2UENNrmWbYNbucfhJFaEcWz"))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [70]:
def deleteConnection(agent, connection_id):
    message = util.buildMessage(
        'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-connections/0.1/delete',
        connection_id=connection_id
    )
    return util.sendMessage(message, agent)

pp.pprint(deleteConnection(connections[1], '31e42ba3-99e8-4404-9003-9a115a4f7153'))


{'@id': '91cdb27f-bc84-4bc3-8359-e6861e35dbb3',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-connections/0.1/ack',
 '~thread': {'thid': 'af76a6a1-3f2f-4b64-b276-ec736bf7b745'}}
